In [3]:
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt
import torch

import nnfabrik
from nnfabrik import builder


import numpy as np
import pickle
import os

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt

import nnvision

from nnfabrik.main import *

In [2]:
%load_ext autoreload
%autoreload 

import datajoint as dj
dj.config['enable_python_native_blobs'] = True


dj.config['database.user']= 'kanderer'
dj.config['database.password']= 'enamel-vendetta-deodorant'


schema_name = 'nnfabrik_monkey_saliency'

schema = dj.schema(schema_name, locals())
dj.config['nnfabrik.schema_name'] = schema_name


Connecting kanderer@134.2.168.16:3306


In [4]:
import datajoint as dj

dj.config['enable_python_native_blobs'] = True

from nnfabrik.templates.trained_model import TrainedModelBase
from nnfabrik.main import *
import os

if not 'stores' in dj.config:
    dj.config['stores'] = {}
    
dj.config['stores']['minio'] = {  # store in s3
    'protocol': 's3',
    'endpoint': os.environ.get('MINIO_ENDPOINT', 'DUMMY_ENDPOINT'),
    'bucket': 'nnfabrik',
    'location': 'dj-store',
    'access_key': os.environ.get('MINIO_ACCESS_KEY', 'FAKEKEY'),
    'secret_key': os.environ.get('MINIO_SECRET_KEY', 'FAKEKEY')
}

In [5]:
basepath = '/data/monkey/toliaslab/CSRF19_V4'
neuronal_data_path = os.path.join(basepath, 'neuronal_data/')
neuronal_data_files = [neuronal_data_path + f for f in listdir(neuronal_data_path) if isfile(join(neuronal_data_path, f))]
image_cache_path = os.path.join(basepath, 'images')

saliency_cache_path = os.path.join(basepath, 'images_saliency')
print(saliency_cache_path)

/data/monkey/toliaslab/CSRF19_V4/images_saliency


In [6]:
@schema
class TrainedModel(TrainedModelBase):
    table_comment = "Trained models"
    storage = "minio"
    model_table = Model
    dataset_table = Dataset
    trainer_table = Trainer
    seed_table = Seed
    user_table = Fabrikant

In [7]:
key = dict(model_hash='cec63aa4435b2a205ec02eafc0a745ee', dataset_hash='ce98e82c2543a503de7611648340380e', trainer_hash = 'f03a6527ab0422767da50e67e2d543ef')

dataloader, model = (TrainedModel & key).load_model()

In [8]:
images = dataloader["train"]["3645713184967"].dataset[:].inputs
responses = dataloader["train"]["3645713184967"].dataset[:].targets

images.shape

torch.Size([11951, 1, 83, 83])

### 1. Get a bunch of Gradient receptive fields

In [ ]:
# I'd start to calculate the RFs only for a bunch of neurons - for 2000 images and 200 Neurons, that cell took me about an hour to compute.

n_images = 100 # 100 images would also be fine
n_neurons = responses.shape[1] # 167 Neurons

RFs = torch.zeros(n_neurons, n_images,*images.shape[1:]) # preallocating RFs

img_x, img_y = images.shape[2:] # x and y dim on the image

print(img_x, img_y)

for i in range(n_images):

    x = torch.rand([1,*images.shape[1::]], device='cpu', requires_grad=True)
    optimizer = torch.optim.SGD([x], lr = 1.0)
    optimizer.zero_grad()
    r = model(x, data_key='3645713184967', pretrained=True)
    for neuron in range(r.shape[1]):
        r[0,neuron].backward(retain_graph=True)
        RFs[neuron, i] = x.grad.data
        x.grad.data.zero_()

83 83


In [ ]:
### Selecting the "best" neurons - with the highest test correlation

In [ ]:
from nnvision.tables.scores import TestCorrelationScore

In [ ]:
(TestCorrelationScore.Units & key).fetch(as_dict=True)

In [ ]:
best_five = (TestCorrelationScore.Units & key).fetch("KEY", "unit_test_correlation", limit=5, order_by="unit_test_correlation DESC")

In [ ]:
neuron_id = [item['unit_id'] for item in best_five[0]]
neuron_id

### 2. Compute the per Neuron Eigenvalues of the img_x * img_y * n_rfs Matrix with kPCA

In [ ]:
rfs_neuron_0 = RFs[8].cpu().detach().numpy().squeeze()

rfs_flattened = np.reshape(rfs_neuron_0,[n_images,-1])
rfs_flattened -= rfs_flattened.mean(axis=1, keepdims=True)

#Kernel matrix 
K = rfs_flattened @ rfs_flattened.T / rfs_flattened.shape[0]
uk, a = eig(K)
uc, a = map(np.real, (uk, a))

#Covariance
C =  rfs_flattened.T @ rfs_flattened / rfs_flattened.shape[0]
uc, v = eig(C)
uk, v = map(np.real, (uc, v))

v2 = rfs_flattened.T @ a
v2 /= np.sqrt( (v2**2).sum(axis=0, keepdims=True)) # normalize each column to length 1 v/||v||_2
v3 = np.reshape(v2,(img_x, img_y, n_images)) # this will be the all eigenvalue RFs. v3[:,:,0] will be the one with the largest component

In [ ]:
with sns.axes_style('ticks'):
    fig, ax = plt.subplots()
    

ax.plot(uc, label='C')
ax.plot(uk, label='K')
ax.set_xlim(0, 100)
sns.despine(trim=True)
fig.legend()

In [ ]:
eigRF = v3[:,:,0]
cAxiMax = np.max(abs(eigRF))
showme = plt.imshow(eigRF.squeeze(), cmap='gray', aspect=1, vmin= -cAxiMax, vmax = cAxiMax)
plt.axis('off')
plt.show()

### 4. Compute Ratio of First and Second Component
for simple cells, we expect that the first PC will be much larger than the second PC (because of phase invariance).
This won't be quite as relevant for mouse data, but why not.

In [ ]:
eig_ratio = []

RFs_np = RFs.cpu().detach().numpy()

print(RFs_np.shape)

for i in neuron_id:
    rfs_flattened = RFs_np[i,::].squeeze()

    rfs_flattened = np.reshape(rfs_flattened,[n_images,-1])
    rfs_flattened -= rfs_flattened.mean(axis=1, keepdims=True)
    K = rfs_flattened @ rfs_flattened.T / rfs_flattened.shape[0]
    uk, a = eig(K)
    uk, a = map(np.real, (uk, a))
    
    #C = rfs_flattened.T @ rfs_flattened / rfs_flattened.shape[0]
    #uc, v = eig(C)
    #uc, v = map(np.real, (uc, v))
    

    print(uk[0])
    eig_ratio.append(uk[1]/uk[0])


In [ ]:
import seaborn as sns
import pandas as pd
sns.set()
x = pd.Series(eig_ratio, name="Ratio of first two PCs")
sns.distplot(x, np.linspace(0,1,20), rug=True)

### Here the reconstructing part is coming

In [ ]:
eigenvalue_matrices = []
for i in neuron_id:
    rfs_flattened = RFs_np[i,::].squeeze()

    rfs_flattened = np.reshape(rfs_flattened,[n_images,-1])
    rfs_flattened -= rfs_flattened.mean(axis=1, keepdims=True)
    K = rfs_flattened @ rfs_flattened.T / rfs_flattened.shape[0]
    uk, a = eig(K)
    uk, a = map(np.real, (uk, a))
    
    #C = rfs_flattened.T @ rfs_flattened / rfs_flattened.shape[0]
    #uc, v = eig(C)
    #uc, v = map(np.real, (uc, v))
    #eig_ratio.append(uk[1]/uk[0])
    
    v2 = rfs_flattened.T @ a
    v2 /= np.sqrt( (v2**2).sum(axis=0, keepdims=True)) # normalize each column to length 1 v/||v||_2
    v3 = np.reshape(v2,(img_x, img_y, n_images)) # this will be the all eigenvalue RFs. v3[:,:,0] will be the one with the largest component
    
    eigenvalue_matrices.append(v3.copy())